#  ECE524 Final Project--Image Mosaicking
#### Ananth Sridhar, Rangapriya Parthasarathy, Song Mei

### Part I: Introduction
Mosaicking is an old art technique where pictures or designs are formed by inlaying small bits of colored stone, glass, or tile. These small bits are visible close up, but the boundaries of the bits will blend and a single recognizable image will show at a distance. 

In the modern digital world, this acient art form has been transformed and combined with new technologies. As one has access to large public image database, individual images, instead of pure-colored blocks can be used as tiles to make pictures. Here is a famous mosaic image of president Obama made by [Anne Savage](http://news.siu.edu/2009/01/011509amh9004.php) during his campaign.
![Obama](http://news.siu.edu/_assets/images/2009/01/asavage_mosaic.jpg)
Nearly 6,000 images of individual faces are used to create this image. Sure enough, it has become a very popular poster because of the special art form. Using image mosaicking to form various pictures is very cool and seems like a daunting task because we are picking and arranging thousands of pictures from a even bigger pool of pictures. There are businesses helping you create your own mosaic work with charge. However, with intuition and just a little knowledge of optimization, we can do it at home ourselves!

We know images consist of pixels, and each pixel can be described using RGB color intensities. In order for two images to look alike, we just need to match their pixels. In this case, for the image to show at a distance, intuitively, we can just treat the basis images as "super pixels" and try to match them with the corresponding subimage of the target image. Naturally, we will take the target image and partition it into a grid whose size is our tile size. Then for each grid, we will choose one image from the basis that resembles the most. The way to qualify "resemblance" is discussed in detail in Part II. Because each chosen basis image resembles the grid best, after processing all grids, we will end up with a mosaicking image that overall best resembles the whole image. Notice that this is the most intuitive way to solve the problem, and it will always achieve the best result. We will present the solution using this method in the first section of Part III. This brute force method is limited by its speed; variants of the algorithm that speeds the process up will be discussed. We will also explore how changing the size of the tiles will affect the performance. The image basis in this project are obtained from online source [mazaika](http://www.mazaika.com/mazdownload.html).
 

### Part II: Mathematical model

From the introduction above, it is clear that image mosaicking is an **assignment problem** which can be modeled as a **Mixed Integer Problem**. The **decision variables** $G_i,i\in\mathbf{I}$ ($\mathbf{I}$ is the set of all grids in target image) are which image (represented by its index $j\in\mathbf{B}$, where $\mathbf{B}$ is the set of basis images of size $s$) to choose for each grid $i$ of the target image. Therefore, $\mathbf{B}$ represent the restricted set of values $G_i$ can take. In the mathematical formulation, additional binary variable for each basis image is introduced to indicate whether this image is chosen for the grid. All these variables satisfy the **SOS constraint**. The **objective** is to minimize the difference$^*$ between the chosen mosaic and the target image. Note the difference function should be linear for the model to be MIP. ($^*$ _Several non-linear functions can be used to describe the resemblance of the grid and the basis image and be implemented within special algorithms. But they fall out of the scope of this course and are not discussed here._)

It is worth pointing out that in the current formulation, the optimization of each grid is independent from one another. Therefore, we can lessen the solvers burden by forming a model that solves the sub-problem of optimizing each grid. To optimize the whole target image, one can simply put the model in a function and call it for each grid of the image. For grid size $m\times n$, target grid can be represented by $m\times n\times 3$ RGB matrix $G$. Before optimization, basis images are pre-processed so that they all have the same size as the grid. Each basis image is represented as $m\times n\times 3$ RGB matrix $B_j,j\in\mathbf{B}$. The difference matrix ($m\times n\times 3$) between target $G$ and candidate $B_j$ is $D_j=G-B_j$.

In this project, we have limited the grid sizes to $2^p\times 2^p,~p\in\{0,1,\ldots,6\}$ square grids for the sake of scaling the basis. Further, we make sure the target image can be divided perfectly into integer number of grids. We have explored several differnt cost functions, and the formulation of each is described below.

#### Models with Different Objective Functions
**1. Sum $L_1$ norm of difference vecor for each pixel.**
$$
minimize:\qquad\sum_{x=1}^m\sum_{y=1}^n\sum_{c=R,G,B} error_{xy,c}\\
subject~to:\qquad error_{xy,c}\geq E[x,y,c]\\
\qquad\qquad error_{xy,c}\geq -E[x,y,c]\\
\qquad\qquad E[x,y,c]=\sum_{j=1}^{s}D_j[x,y,c]\cdot p_j\\
\qquad\qquad \sum_{j=1}^s p_j=1, p_j\in\{0,1\}\\
$$


**2. $L_1$ norm of average difference vector.**
$$
minimize:\qquad\sum_{c=R,G,B} error_{c}\\
subject~to:\qquad error_{c}\geq E[c]\\
\qquad\qquad error_{c}\geq -E[c]\\
\qquad\qquad E[c]=\sum_{j=1}^{s}D_j[c]\cdot p_j\\
D_j[c]=\frac{1}{mn}\sum_{x=1}^m\sum_{y=1}^nG[x,y,c]-\frac{1}{mn}\sum_{x=1}^m\sum_{y=1}^nB_j[x,y,c],c\in\{R,G,B\}\\
\qquad\qquad \sum_{j=1}^{s}p_{j}=1, p_j\in\{0,1\}
$$

**3. $L_1$ norm of difference vector between the most frequent color intensity.**
$$
minimize:\qquad\sum_{c=R,G,B} error_{c}\\
subject~to:\qquad error_{c}\geq E[c]\\
\qquad\qquad error_{c}\geq -E[c]\\
\qquad\qquad E[c]=\sum_{j=1}^{s}D_j[c]\cdot p_j\\
\qquad\qquad D_j[c]=mode(G[:,:,c])-mode(B_j[:,:,c]),c=\{R,G,B\}\\
\qquad\qquad \sum_{j=1}^{s}p_{j}=1, p_j\in\{0,1\}
$$


$$$$
The first formulation is the most straight forward yet most expensive way. The number of varibles and constraints scales with the size of the target image. Further, by trying to match individual pixels, one may not necessarily get the best performance because mosaicking is more about overall resemblance at a distance. As a result, the optimization model using this objective fuction is not shown in the solution part.

Considering matching individual pixels is very expensive and all we care about is the images look close at a distance. It is straight forward to treat each basis image as a whole. We can extract features from the picture. Two most frequently used are the mean (average) of the pixel RGB intensities and the mode (most frequent value) of the pixel RGB. Both of these are implemented.

### Part III: Solution

### Part IV: Results and Discussion

The first formulation is the most straight forward yet most expensive way. The number of varibles and constraints scales with the size of the target image. Further, by trying to match individual pixels, one may not necessarily get the best performance because mosaicking is more about overall resemblance at a distance. As a result, the optimization model using this objective fuction is not shown in the solution part. Inspired by this, we can treat each basis as a whole.
The most frequent color intensity represents how the image looks in general (mostly red or mostly purple), therefore is a pretty good indicator for two images to look alike at a distance. 

### Part V: Conclusion